Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

from drive.MyDrive.CodeFiles.SERCode.Code.features1 import *
from drive.MyDrive.CodeFiles.SERCode.Code.helper import *

!pip install audiosegment
!sudo apt-get install libportaudio2
!pip install sounddevice
!apt-get install -y sox

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import audiosegment
import sounddevice as sd

import os
import sys
import wave
import copy
import math
import pickle

from scipy import signal
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Embedding, Conv1D, Conv2D, Dropout, MaxPooling2D, Bidirectional
from keras.layers import TimeDistributed
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import label_binarize



MessageError: ignored

IEMOCAP DATA COLLECTION

In [ ]:
batch_size = 64
nb_feat = 34
nb_class = 4
nb_epoch = 80

optimizer = 'Adadelta'


emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = "/content/drive/MyDrive/Data/"
sessions = ['Session1','Session2','Session3','Session4','Session5']
framerate = 16000

def get_mocap_rot(path_to_mocap_rot, filename, start,end):
    f = open(path_to_mocap_rot + filename, 'r').read()
    f = np.array(f.split('\n'))
    mocap_rot = []
    mocap_rot_avg = []
    f = f[2:]
    counter = 0
    for data in f:
        counter+=1
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        if(float(data2[1])>start and float(data2[1])<end):
            mocap_rot_avg.append(np.array(data2[2:]).astype(float))

    mocap_rot_avg = np.array_split(np.array(mocap_rot_avg), 200)
    for spl in mocap_rot_avg:
        mocap_rot.append(np.mean(spl, axis=0))
    return np.array(mocap_rot)

def get_mocap_hand(path_to_mocap_hand, filename, start,end):
    f = open(path_to_mocap_hand + filename, 'r').read()
    f = np.array(f.split('\n'))
    mocap_hand = []
    mocap_hand_avg = []
    f = f[2:]
    counter = 0
    for data in f:
        counter+=1
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        if(float(data2[1])>start and float(data2[1])<end):
            mocap_hand_avg.append(np.array(data2[2:]).astype(float))

    mocap_hand_avg = np.array_split(np.array(mocap_hand_avg), 200)
    for spl in mocap_hand_avg:
        mocap_hand.append(np.mean(spl, axis=0))
    return np.array(mocap_hand)

def get_mocap_head(path_to_mocap_head, filename, start,end):
    f = open(path_to_mocap_head + filename, 'r').read()
    f = np.array(f.split('\n'))
    mocap_head = []
    mocap_head_avg = []
    f = f[2:]
    counter = 0
    for data in f:
        counter+=1
        data2 = data.split(' ')
        if(len(data2)<2):
            continue
        if(float(data2[1])>start and float(data2[1])<end):
            mocap_head_avg.append(np.array(data2[2:]).astype(float))

    mocap_head_avg = np.array_split(np.array(mocap_head_avg), 200)
    for spl in mocap_head_avg:
        mocap_head.append(np.mean(spl, axis=0))
    return np.array(mocap_head)



def read_iemocap_mocap():
    data = []
    ids = {}
    for session in sessions:
        path_to_wav = data_path + session + '/dialog/wav/'
        path_to_emotions = data_path +session + '/dialog/EmoEvaluation/'
        path_to_transcriptions = data_path  + session + '/dialog/transcriptions/'
        path_to_mocap_hand = data_path + session + '/dialog/MOCAP_hand/'
        path_to_mocap_rot = data_path + session + '/dialog/MOCAP_rotated/'
        path_to_mocap_head = data_path + session + '/dialog/MOCAP_head/'

        files2 = os.listdir(path_to_wav)

        files = []
        for f in files2:
            if f.endswith(".wav"):
                if f[0] == '.':
                    files.append(f[2:-4])
                else:
                    files.append(f[:-4])


        for f in files:
            print(f)
            mocap_f = f
            if (f== 'Ses05M_script01_1b'):
                mocap_f = 'Ses05M_script01_1'

            wav = get_audio(path_to_wav, f + '.wav')
            transcriptions = get_transcriptions(path_to_transcriptions , f + '.txt')

            emotions = get_emotions(path_to_emotions, f + '.txt')
            sample = split_wav(wav, emotions)

            for ie, e in enumerate(emotions):
                '''if 'F' in e['id']:
                    e['signal'] = sample[ie]['left']
                else:
                    e['signal'] = sample[ie]['right']'''

                e['signal'] = sample[ie]['left']
                e.pop("left", None)
                e.pop("right", None)
                e['transcription'] = transcriptions[e['id']]
                e['mocap_hand'] = get_mocap_hand(path_to_mocap_hand, mocap_f + '.txt', e['start'], e['end'])
                e['mocap_rot'] = get_mocap_rot(path_to_mocap_rot, mocap_f + '.txt', e['start'], e['end'])
                e['mocap_head'] = get_mocap_head(path_to_mocap_head, mocap_f + '.txt', e['start'], e['end'])
                if e['emotion'] in emotions_used:
                    if e['id'] not in ids:
                        data.append(e)
                        ids[e['id']] = 1


    sort_key = get_field(data, "id")
    return np.array(data)[np.argsort(sort_key)]

data = read_iemocap_mocap()

import pickle
with open(data_path + '/./'+'iemocap_data_collected.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

Ses01M_impro06
Ses01M_script02_2
Ses01F_script01_1
Ses01F_script03_2
Ses01M_script01_1
Ses01M_impro02
Ses01M_script01_3
Ses01F_script01_2
Ses01M_impro03
Ses01F_impro06
Ses01M_script01_2
Ses01M_impro01
Ses01F_script03_1
Ses01F_script02_2
Ses01M_impro04
Ses01F_impro03
Ses01F_script02_1
Ses01F_impro01
Ses01M_script03_1
Ses01M_impro05
Ses01F_impro05
Ses01F_impro02
Ses01M_script03_2
Ses01F_script01_3
Ses01F_impro07
Ses01M_script02_1
Ses01M_impro07
Ses01F_impro04
Ses02M_impro01
Ses02F_script03_2
Ses02F_impro05
Ses02M_script03_1
Ses02F_impro03
Ses02F_impro08
Ses02M_impro03
Ses02M_impro06
Ses02F_impro04
Ses02F_impro06
Ses02F_script03_1
Ses02F_script02_1
Ses02M_script01_3
Ses02F_impro07
Ses02M_script02_1
Ses02F_script01_2
Ses02F_script01_3
Ses02F_impro02
Ses02M_script02_2
Ses02M_script01_2
Ses02M_impro02
Ses02M_impro07
Ses02M_impro04
Ses02F_script01_1
Ses02F_script02_2
Ses02M_impro08
Ses02M_script03_2
Ses02M_impro05
Ses02M_script01_1
Ses02F_impro01
Ses03F_script02_1
Ses03M_script03_2
Ses03F_imp

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Ses05F_impro04
Ses05M_impro05
Ses05F_script03_2
Ses05F_impro03
Ses05M_script03_1
Ses05F_script02_2
Ses05F_impro02
Ses05F_script03_1
Ses05M_script03_2
Ses05F_impro05
Ses05M_script01_1
Ses05F_script01_3


VOICED FEATURES

In [ ]:
batch_size = 64
nb_feat = 34
nb_class = 4


optimizer = 'Adadelta'


emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = "/content/drive/MyDrive/Data/"
sessions = ['Session1','Session2','Session3','Session4','Session5']
framerate = 16000

NameError: ignored

In [ ]:
with open(data_path +'/data_collected2.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

# read output label
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
Y = label_binarize(y=Y, classes=emotions_used)
Y

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [ ]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if len(st_f.shape) > 1 and st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1

        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif len(st_f.shape) > 1 and st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

# doing silence removal (already saved as npy)
voiced_feat = []
for ses_mod in data2:
    x_head = ses_mod['signal']
    seg = audiosegment.from_numpy_array(x_head, framerate)
    voice = seg.filter_silence(0.01, 0.01)
    st_features = calculate_features(voice.to_numpy_array(), framerate, None)
    if st_features.shape[0] == 0:
      print("Empty st_features array, skipping this segment")
      continue  # or return or handle the case in an appropriate way

    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    voiced_feat.append(st_features.T)

voiced_feat = np.array(voiced_feat)
voiced_feat.shape

(4936, 100, 34)

In [ ]:
np.save('voiced_feat.npy', voiced_feat)

TEST TRAIN SPLIT

In [ ]:
X= np.load('/content/drive/MyDrive/Data/voiced_feat.npy')


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)



MODEL 1 LSTM

In [ ]:
def speech_model1():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(100, 34)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

model1 = speech_model1()
model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1120256   
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
 activation_1 (Activation)   (None, 4)                 0         
                                                                 
Total params: 2,041,348
Trainable params: 2,041,348
Non-

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model1.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq='epoch')
hist1 = model1.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1, shuffle = False,callbacks=[checkpoint])

Epoch 1/50
104/104 [==============================] - ETA: 0s - loss: 1.3556 - accuracy: 0.3662
Epoch 1: loss improved from inf to 1.35559, saving model to best_model1.hdf5
104/104 [==============================] - 155s 1s/step - loss: 1.3556 - accuracy: 0.3662
Epoch 2/50
104/104 [==============================] - ETA: 0s - loss: 1.3530 - accuracy: 0.3656
Epoch 2: loss improved from 1.35559 to 1.35304, saving model to best_model1.hdf5
104/104 [==============================] - 152s 1s/step - loss: 1.3530 - accuracy: 0.3656
Epoch 3/50
104/104 [==============================] - ETA: 0s - loss: 1.3494 - accuracy: 0.3735
Epoch 3: loss improved from 1.35304 to 1.34941, saving model to best_model1.hdf5
104/104 [==============================] - 152s 1s/step - loss: 1.3494 - accuracy: 0.3735
Epoch 4/50
104/104 [==============================] - ETA: 0s - loss: 1.3712 - accuracy: 0.3671
Epoch 4: loss did not improve from 1.34941
104/104 [==============================] - 153s 1s/step - loss: 

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report

Y_pred=model1.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)
print('\nConfusion Matrix\n')
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

acc1 = hist1.history['accuracy']
print('\nTraining Accuracy\n')
print('Mean Training Accuracy',np.mean(acc1))
print('Max Training Accuracy',max(acc1))

print('\nClassification Report\n')
print(classification_report(Y_test, Y_pred, target_names= emotions_used))

print('Class Wise Accuracy\n')
acc=0
for idx in range(0,4):
  sum=0
  for j in range(0,4):
    sum= sum + cm[idx][j]
  acc = cm[idx][idx] / sum
  print(emotions_used[idx]," : ",acc)

51/51 [==============================] - 18s 360ms/step

Confusion Matrix

[[109  84 144  37]
 [ 43  94 156  51]
 [ 17  59 300 175]
 [  4   4  49 303]]

Training Accuracy

Mean Training Accuracy 0.4784941053390503
Max Training Accuracy 0.599334716796875

Classification Report

              precision    recall  f1-score   support

         ang       0.63      0.29      0.40       374
         exc       0.39      0.27      0.32       344
         neu       0.46      0.54      0.50       551
         sad       0.54      0.84      0.65       360

    accuracy                           0.49      1629
   macro avg       0.50      0.49      0.47      1629
weighted avg       0.50      0.49      0.47      1629

Class Wise Accuracy

ang  :  0.2914438502673797
exc  :  0.27325581395348836
neu  :  0.5444646098003629
sad  :  0.8416666666666667


MODEL 2 LSTM GLOBAL ATTENTION

In [ ]:
from keras import backend as K
from keras.layers import LSTM,RNN
from keras import regularizers, constraints, initializers, activations
from tensorflow.keras.layers import InputSpec,Layer
from tensorflow.keras.layers import LSTMCell

class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x




In [ ]:
def global_attention_model(optimizer='adam'):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(100, 34)))
    model.add(GlobalSelfAttention(num_heads=4,key_dim=128))
    model.add(FeedForward(512,2048))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model2 = global_attention_model()
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 100, 512)         595968    
 l)                                                              
                                                                 
 global_self_attention (Glob  (None, 100, 512)         1051648   
 alSelfAttention)                                                
                                                                 
 feed_forward (FeedForward)  (None, 100, 512)          2100736   
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                3276864   
                                                                 
 batch_normalization (BatchN  (None, 64)              

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model2.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto',  save_freq='epoch')
hist2 = model2.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1, shuffle = False,
                 callbacks=[checkpoint])

Epoch 1/50
104/104 [==============================] - ETA: 0s - loss: 1.2699 - accuracy: 0.4064
Epoch 1: loss improved from inf to 1.26987, saving model to best_model2.hdf5
104/104 [==============================] - 173s 2s/step - loss: 1.2699 - accuracy: 0.4064
Epoch 2/50
104/104 [==============================] - ETA: 0s - loss: 1.1466 - accuracy: 0.4651
Epoch 2: loss improved from 1.26987 to 1.14660, saving model to best_model2.hdf5
104/104 [==============================] - 163s 2s/step - loss: 1.1466 - accuracy: 0.4651
Epoch 3/50
104/104 [==============================] - ETA: 0s - loss: 1.1458 - accuracy: 0.4681
Epoch 3: loss improved from 1.14660 to 1.14585, saving model to best_model2.hdf5
104/104 [==============================] - 166s 2s/step - loss: 1.1458 - accuracy: 0.4681
Epoch 4/50
104/104 [==============================] - ETA: 0s - loss: 1.1530 - accuracy: 0.4738
Epoch 4: loss did not improve from 1.14585
104/104 [==============================] - 161s 2s/step - loss: 

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report

Y_pred=model2.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)
print('\nConfusion Matrix\n')

cm = confusion_matrix(Y_test, Y_pred)
print(cm)
print('\nTraining Accuracy\n',)
acc2 = hist2.history['accuracy']
print('Mean Training Accuracy',np.mean(acc2))
print('Max Training Accuracy',max(acc2))



print('\nClassification Report\n')
print(classification_report(Y_test, Y_pred, target_names= emotions_used))


# Calculate the accuracy for each one of our classes
print('Class Wise Accuracy\n')
acc=0
for idx in range(0,4):
  sum=0
  for j in range(0,4):
    sum= sum + cm[idx][j]
  acc = cm[idx][idx] / sum
  print(emotions_used[idx]," : ",acc)

51/51 [==============================] - 24s 469ms/step

Confusion Matrix

[[113 111  96  54]
 [ 28 173  99  44]
 [ 36  93 265 157]
 [  7  12  54 287]]

Training Accuracy

Mean Training Accuracy 0.7087390398979188
Max Training Accuracy 0.9634109735488892

Classification Report

              precision    recall  f1-score   support

         ang       0.61      0.30      0.41       374
         exc       0.44      0.50      0.47       344
         neu       0.52      0.48      0.50       551
         sad       0.53      0.80      0.64       360

    accuracy                           0.51      1629
   macro avg       0.53      0.52      0.50      1629
weighted avg       0.53      0.51      0.50      1629

Class Wise Accuracy

ang  :  0.30213903743315507
exc  :  0.502906976744186
neu  :  0.4809437386569873
sad  :  0.7972222222222223


MODEL 3 SIMPLE LSTM ATTENTION

In [ ]:
from keras import backend as K
from keras.layers import LSTM,RNN
from keras import regularizers, constraints, initializers, activations
from tensorflow.keras.layers import InputSpec,Layer
from tensorflow.keras.layers import LSTMCell

# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context


In [ ]:
from keras.layers import Input, Dense, SimpleRNN, LSTM, Bidirectional
def LSTMattention(hidden_units, dense_units, input_shape, activation):
    mainmodel=Sequential()
    x=Input(shape=input_shape)
    BiLSTM_layer = Bidirectional(LSTM(hidden_units, return_sequences=True))(x)
    attention_layer = attention()(BiLSTM_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    mainmodel.add(model)
    mainmodel.add(Dense(512, activation='relu'))
    mainmodel.add(Dense(4, activation='softmax'))

    mainmodel.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return mainmodel

model3 = LSTMattention(hidden_units=128, dense_units=4,input_shape=(100,34), activation='tanh')
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 4)                 168296    
                                                                 
 dense_5 (Dense)             (None, 512)               2560      
                                                                 
 dense_6 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 172,908
Trainable params: 172,908
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model3.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto',  save_freq='epoch')
hist3 = model3.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1, shuffle = False,
                 callbacks=[checkpoint])

Epoch 1/50
104/104 [==============================] - ETA: 0s - loss: 1.2451 - accuracy: 0.4276
Epoch 1: loss improved from inf to 1.24513, saving model to best_model3.hdf5
104/104 [==============================] - 42s 293ms/step - loss: 1.2451 - accuracy: 0.4276
Epoch 2/50
104/104 [==============================] - ETA: 0s - loss: 1.1702 - accuracy: 0.4599
Epoch 2: loss improved from 1.24513 to 1.17022, saving model to best_model3.hdf5
104/104 [==============================] - 31s 296ms/step - loss: 1.1702 - accuracy: 0.4599
Epoch 3/50
104/104 [==============================] - ETA: 0s - loss: 1.1396 - accuracy: 0.4681
Epoch 3: loss improved from 1.17022 to 1.13960, saving model to best_model3.hdf5
104/104 [==============================] - 31s 298ms/step - loss: 1.1396 - accuracy: 0.4681
Epoch 4/50
104/104 [==============================] - ETA: 0s - loss: 1.1161 - accuracy: 0.4871
Epoch 4: loss improved from 1.13960 to 1.11607, saving model to best_model3.hdf5
104/104 [===========

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report

Y_pred=model3.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)
cm = confusion_matrix(Y_test, Y_pred)
print('\nConfusion Matrix\n')
print(cm)

acc3 = hist3.history['accuracy']
print('\nTraining Accuracy\n',)
print('Mean Training Accuracy',np.mean(acc3))
print('Max Training Accuracy',max(acc3))

print('\nClassification Report\n')
print(classification_report(Y_test, Y_pred, target_names= emotions_used))

# Calculate the accuracy for each one of our classes

print('Class Wise Accuracy\n')
acc=0
for idx in range(0,4):
  sum=0
  for j in range(0,4):
    sum= sum + cm[idx][j]
  acc = cm[idx][idx] / sum
  print(emotions_used[idx]," : ",acc)

51/51 [==============================] - 9s 169ms/step

Confusion Matrix

[[181  70 116   7]
 [ 50 141 133  20]
 [ 50  56 348  97]
 [ 11  16  79 254]]

Training Accuracy

Mean Training Accuracy 0.6290232855081558
Max Training Accuracy 0.8088902235031128

Classification Report

              precision    recall  f1-score   support

         ang       0.62      0.48      0.54       374
         exc       0.50      0.41      0.45       344
         neu       0.51      0.63      0.57       551
         sad       0.67      0.71      0.69       360

    accuracy                           0.57      1629
   macro avg       0.58      0.56      0.56      1629
weighted avg       0.57      0.57      0.56      1629

Class Wise Accuracy

ang  :  0.4839572192513369
exc  :  0.40988372093023256
neu  :  0.631578947368421
sad  :  0.7055555555555556


MODEL 3 BUT SAME

In [ ]:
from keras.layers import Input, Dense, SimpleRNN, LSTM, Bidirectional
def LSTMattention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    BiLSTM_layer = Bidirectional(LSTM(hidden_units, return_sequences=True))(x)
    attention_layer = attention()(BiLSTM_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    dense_layer1=Dense(512, activation='relu')(outputs)
    dense_layer2 = Dense(4, activation='softmax')(dense_layer1)
    model=Model(x,dense_layer2)


    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

model31 = LSTMattention(hidden_units=128, dense_units=4,input_shape=(100,34), activation='tanh')
model31.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 34)]         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 100, 256)         166912    
 nal)                                                            
                                                                 
 attention_2 (attention)     (None, 256)               356       
                                                                 
 dense_7 (Dense)             (None, 4)                 1028      
                                                                 
 dense_8 (Dense)             (None, 512)               2560      
                                                                 
 dense_9 (Dense)             (None, 4)                 2052      
                                                           

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model31.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto',  save_freq='epoch')
hist3 = model31.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1, shuffle = False,
                 callbacks=[checkpoint])

Epoch 1/50
104/104 [==============================] - ETA: 0s - loss: 1.1463 - accuracy: 0.4708
Epoch 1: loss improved from inf to 1.14628, saving model to best_model31.hdf5
104/104 [==============================] - 41s 392ms/step - loss: 1.1463 - accuracy: 0.4708
Epoch 2/50
104/104 [==============================] - ETA: 0s - loss: 1.1299 - accuracy: 0.4766
Epoch 2: loss improved from 1.14628 to 1.12994, saving model to best_model31.hdf5
104/104 [==============================] - 30s 289ms/step - loss: 1.1299 - accuracy: 0.4766
Epoch 3/50
104/104 [==============================] - ETA: 0s - loss: 1.1183 - accuracy: 0.4859
Epoch 3: loss improved from 1.12994 to 1.11828, saving model to best_model31.hdf5
104/104 [==============================] - 31s 302ms/step - loss: 1.1183 - accuracy: 0.4859
Epoch 4/50
104/104 [==============================] - ETA: 0s - loss: 1.1070 - accuracy: 0.4995
Epoch 4: loss improved from 1.11828 to 1.10695, saving model to best_model31.hdf5
104/104 [=======

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report

Y_pred=model31.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)

cm = confusion_matrix(Y_test, Y_pred)
print('\nConfusion Matrix\n')
print(cm)

acc3 = hist3.history['accuracy']
print('\nTraining Accuracy\n',)
print('Mean Training Accuracy',np.mean(acc3))
print('Max Training Accuracy',max(acc3))

print('\nClassification Report\n')
print(classification_report(Y_test, Y_pred, target_names= emotions_used))

# Calculate the accuracy for each one of our classes

print('Class Wise Accuracy\n')
acc=0
for idx in range(0,4):
  sum=0
  for j in range(0,4):
    sum= sum + cm[idx][j]
  acc = cm[idx][idx] / sum
  print(emotions_used[idx]," : ",acc)

51/51 [==============================] - 10s 201ms/step

Confusion Matrix

[[177  94  89  14]
 [ 56 203  68  17]
 [ 43 134 291  83]
 [ 16  31  74 239]]

Training Accuracy

Mean Training Accuracy 0.6717024451494217
Max Training Accuracy 0.860296368598938

Classification Report

              precision    recall  f1-score   support

         ang       0.61      0.47      0.53       374
         exc       0.44      0.59      0.50       344
         neu       0.56      0.53      0.54       551
         sad       0.68      0.66      0.67       360

    accuracy                           0.56      1629
   macro avg       0.57      0.56      0.56      1629
weighted avg       0.57      0.56      0.56      1629

Class Wise Accuracy

ang  :  0.4732620320855615
exc  :  0.5901162790697675
neu  :  0.5281306715063521
sad  :  0.6638888888888889


MODEL 4 WITHOUT OUTPUT LAYER

In [ ]:
from keras.layers import Input, Dense, SimpleRNN, LSTM, Bidirectional
def LSTM_attention_NoDense4(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    BiLSTM_layer = Bidirectional(LSTM(hidden_units, return_sequences=True))(x)
    attention_layer = attention()(BiLSTM_layer)
    dense_layer1=Dense(512, activation='relu')(attention_layer)
    dense_layer2 = Dense(4, activation='softmax')(dense_layer1)
    model=Model(x,dense_layer2)


    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

model4 = LSTM_attention_NoDense4(hidden_units=128, dense_units=4,input_shape=(100,34), activation='tanh')
model4.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 34)]         0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 100, 256)         166912    
 nal)                                                            
                                                                 
 attention_3 (attention)     (None, 256)               356       
                                                                 
 dense_10 (Dense)            (None, 512)               131584    
                                                                 
 dense_11 (Dense)            (None, 4)                 2052      
                                                                 
Total params: 300,904
Trainable params: 300,904
Non-trainable params: 0
_____________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model4.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto',  save_freq='epoch')
hist4 = model4.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1, shuffle = False,
                 callbacks=[checkpoint])

Epoch 1/50
104/104 [==============================] - ETA: 0s - loss: 1.2806 - accuracy: 0.4034
Epoch 1: loss improved from inf to 1.28064, saving model to best_model4.hdf5
104/104 [==============================] - 35s 291ms/step - loss: 1.2806 - accuracy: 0.4034
Epoch 2/50
104/104 [==============================] - ETA: 0s - loss: 1.1676 - accuracy: 0.4530
Epoch 2: loss improved from 1.28064 to 1.16763, saving model to best_model4.hdf5
104/104 [==============================] - 30s 290ms/step - loss: 1.1676 - accuracy: 0.4530
Epoch 3/50
104/104 [==============================] - ETA: 0s - loss: 1.1363 - accuracy: 0.4751
Epoch 3: loss improved from 1.16763 to 1.13633, saving model to best_model4.hdf5
104/104 [==============================] - 30s 293ms/step - loss: 1.1363 - accuracy: 0.4751
Epoch 4/50
104/104 [==============================] - ETA: 0s - loss: 1.1144 - accuracy: 0.4890
Epoch 4: loss improved from 1.13633 to 1.11439, saving model to best_model4.hdf5
104/104 [===========

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report

Y_pred=model4.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)

cm = confusion_matrix(Y_test, Y_pred)
print('\nConfusion Matrix\n')
print(cm)

acc4 = hist4.history['accuracy']
print('\nTraining Accuracy\n',)
print('Mean Training Accuracy',np.mean(acc4))
print('Max Training Accuracy',max(acc4))

print('\nClassification Report\n')
print(classification_report(Y_test, Y_pred, target_names= emotions_used))

# Calculate the accuracy for each one of our classes

print('Class Wise Accuracy\n')
acc=0
for idx in range(0,4):
  sum=0
  for j in range(0,4):
    sum= sum + cm[idx][j]
  acc = cm[idx][idx] / sum
  print(emotions_used[idx]," : ",acc)

51/51 [==============================] - 9s 175ms/step

Confusion Matrix

[[166  87 114   7]
 [ 53 172 100  19]
 [ 50  85 349  67]
 [ 13  21 100 226]]

Training Accuracy

Mean Training Accuracy 0.7194980347156524
Max Training Accuracy 0.942243754863739

Classification Report

              precision    recall  f1-score   support

         ang       0.59      0.44      0.51       374
         exc       0.47      0.50      0.49       344
         neu       0.53      0.63      0.57       551
         sad       0.71      0.63      0.67       360

    accuracy                           0.56      1629
   macro avg       0.57      0.55      0.56      1629
weighted avg       0.57      0.56      0.56      1629

Class Wise Accuracy

ang  :  0.44385026737967914
exc  :  0.5
neu  :  0.6333938294010889
sad  :  0.6277777777777778
